# Double pendulum

In [ ]:
using DifferentialEquations
using Plots

### Equations of motion

Symmetric double pendulum

$$
\mathcal{L} = \dot{\theta_1}^2 + \dfrac{1}{2}\dot{\theta_2}^2 + \dot{\theta_1}\dot{\theta_2}\cos(\theta_1 - \theta_2) + 2\cos\theta_1 + \cos\theta_2
$$

$$
\begin{cases}
& \ddot{\theta_1} & = & \dfrac{
\phantom{2}\sin\theta_2\cos(\theta_1 - \theta_2) -
2\sin\theta_1 -
\sin(\theta_1 - \theta_2)
\Big[ \phantom{2}\dot{\theta_2}^2 + \cos(\theta_1 - \theta_2)\dot{\theta_1}^2 \Big]
}{2 - \cos^2(\theta_1 - \theta_2)} & \\
& \ddot{\theta_2} & = & \dfrac{
2\sin\theta_1\cos(\theta_1 - \theta_2) -
2\sin\theta_2 +
\sin(\theta_1 - \theta_2)
\Big[ 2\dot{\theta_1}^2 + \cos(\theta_1 - \theta_2)\dot{\theta_2}^2 \Big]
}{2 - \cos^2(\theta_1 - \theta_2)}
\end{cases}
$$

In [ ]:
function dpendulum(du, u, p, t)
    sdiff = sin(u[1] - u[2])
    cdiff = cos(u[1] - u[2])
    detM = 2 - cdiff^2
    du[1] = u[3]
    du[2] = u[4]
    du[3] = (sin(u[2])*cdiff - 2*sin(u[1]) - sdiff*(u[4]^2 + cdiff*u[3]^2)) / detM
    du[4] = (2*sin(u[1])*cdiff -2*sin(u[2]) + sdiff*(2*u[3]^2 + cdiff*u[4]^2)) / detM
end

### Constant of motion

In [ ]:
function energy(u)
    e = u[3]^2 + (1/2)*u[4]^2 + u[3]*u[4]*cos(u[1] - u[2]) + 3 - 2*cos(u[1]) - cos(u[2])
end

### Time span and initial conditions

In [ ]:
t_max = 384.0
t_step = 0.02

u0 = [0, pi/2, 0.0, 0.0]
#u0 = [3*pi/12, -acos(1.0 - 2*cos(3*pi/12)), 0, 0]
energy(u0)

### Numerical solution

In [ ]:
prob = ODEProblem(dpendulum, u0, (0.0, t_max))
sol = solve(prob, Vern7(), adaptive=false, dt=t_step)
data = reduce(hcat, sol.u)'

### Orbit $(\theta_1, \dot{\theta_1})$

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotOrbit = scatter(
    data[:,1], data[:,3],
    aspect_ratio = 1,
    xlabel = "\\theta_1",
    ylabel = "d \\theta_1 / d t",
    plot_title = "Double pendulum",
    markersize = 0.4,
    alpha = 0.15,
    legend = false,
    dpi = 300
)
annotate!(
    xlims(plotOrbit)[1] + 0.12*(xlims(plotOrbit)[2] - xlims(plotOrbit)[1]),
    ylims(plotOrbit)[2] - 0.03*(ylims(plotOrbit)[2] - ylims(plotOrbit)[1]),
    text("E = $(round(energy(u0), digits=4))", :red, :right, 4)
)

### Integration error

In [ ]:
Plots.scalefontsizes()
Plots.scalefontsizes(0.5)
plotEnergy = scatter(
    sol.t,
    map(eachrow(data)) do s
        energy(s) - energy(u0)
    end,
    xlabel = "t",
    ylabel = "Energy error - \\Delta E",
    plot_title = "Double pendulum",
    markersize = 0.5,
    alpha = 0.25,
    legend = false,
    dpi = 300
)